# Praca domowa nr 4
Jakub Szypuła

## Załadowanie danych i pakietów

### Apartments

In [1]:
import pandas as pd
import numpy as np
apartments = pd.read_csv('https://raw.githubusercontent.com/mini-pw/2020L-WUM/master/Prace_domowe/Praca_domowa4/Grupa1/apartments.csv')
apartments.head()

,m2.price,construction.year,surface,floor,no.rooms,district
0,5897,1953,25,3,1,Srodmiescie
1,1818,1992,143,9,5,Bielany
2,3643,1937,56,1,2,Praga
3,3517,1995,93,7,3,Ochota
4,3013,1992,144,6,5,Mokotow


### AoE2 Voobly Data

Jako drugi zbiór danych postanowiłem wybrać zbiór danych dotyczący meczy 1 na 1 w grze strategicznej Age of Empires II, rozgrywanej przez platformę Voobly. Jako zmienną celu przyjąłem `player_1_wins` oznaczającą zwycięstwo gracza nr 1.

Ze względu na ogromny rozmiar danych, postanowiłem je określić do losowych 1000 obserwacji.

Źródło danych: https://blog.macuyiko.com/post/2018/predicting-voobly-age-of-empires-2-matches.html

In [2]:
aoe2 = pd.read_csv('https://github.com/Macuyiko/aoe2predict/raw/master/prepared_data.csv.gz')

In [3]:
aoe2.shape

(1808535, 67)

In [4]:
aoe2 = aoe2.sample(n=1000, random_state=11)
aoe2.shape

(1000, 67)

In [5]:
aoe2.head()

,match_id,current_time,diff_units_infantry,diff_units_ranged,diff_units_siege,diff_units_war_ships,diff_units_cavalry_archers,diff_units_cavalry,diff_units_monks,diff_units_idle_fishing_ships,...,diff_worth_lost,diff_score_military,diff_score_economy,diff_score_technology,diff_score_society,diff_score_total,diff_kills,diff_razes,player_1_wins,diff_rating
235862,16721218,1060026,3,12,0,0,0,-1,0,0,...,-160,32,115,-20,0,127,1,0,1,-32
1661235,16762423,1405152,0,0,1,0,4,2,1,0,...,-459,36,101,-5,0,132,2,0,0,-53
1270841,16724135,3503286,8,22,0,0,0,-7,-1,0,...,3446,-345,178,-976,98,-1045,65,5,0,32
1675612,16768685,561606,-3,0,0,0,0,1,0,0,...,-80,16,-24,110,0,102,1,0,1,17
988008,16723315,979056,0,0,0,0,0,3,0,0,...,-50,0,298,0,0,298,0,0,0,211


In [6]:
aoe2braki=aoe2.isnull().mean() 
aoe2braki.sum()

0.0

Na szczęście w zbiorze danych nie ma żadnych braków.

In [7]:
from sklearn.model_selection import train_test_split
y_ap = apartments[["district"]]
X_ap = apartments.drop(["district"], axis=1)
X_train_ap, X_test_ap, y_train_ap, y_test_ap = train_test_split(X_ap, y_ap, test_size=0.15, random_state=111)
y_train_ap = y_train_ap.values.ravel()
y_test_ap = y_test_ap.values.ravel()
y_aoe2 = aoe2[["player_1_wins"]]
X_aoe2 = aoe2.drop(["player_1_wins"], axis=1)
X_train_aoe2, X_test_aoe2, y_train_aoe2, y_test_aoe2 = train_test_split(X_aoe2, y_aoe2, test_size = 0.15, random_state = 111)
y_train_aoe2 = y_train_aoe2.values.ravel()
y_test_aoe2 = y_test_aoe2.values.ravel()

## SVM bez skalowania

In [8]:
from sklearn import svm
from sklearn.metrics import accuracy_score
model = svm.SVC()

### Apartments

In [8]:
model.fit(X_train_ap, y_train_ap)
y_predicted_ap = model.predict(X_test_ap)
accuracy_score(y_test_ap, y_predicted_ap)

0.20666666666666667

#### Strojenie


In [14]:
from sklearn.model_selection import RandomizedSearchCV

C = np.arange(0.1, 15, 0.1)
degree = np.arange(1, 6, 1)
gamma = ["scale", "auto"] + np.arange(0.1, 0.5, 0.1).tolist()
hyperF = dict(C = C, degree = degree, gamma = gamma)

grid = RandomizedSearchCV(model, hyperF, cv = 11, random_state = 11)
grid.fit(X_train_ap, y_train_ap)
grid.best_params_

{'gamma': 'auto', 'degree': 5, 'C': 6.5}

In [45]:
svm_ap_strj = svm.SVC(gamma = 0.11, degree = 4, C = 14.5)
svm_ap_strj.fit(X_train_ap, y_train_ap)
y_predicted_ap_strj = svm_ap_strj.predict(X_test_ap)
accuracy_score(y_test_ap, y_predicted_ap_strj)

0.06666666666666667

Automatyczne strojenie znacząco pogorszyło wynik. Być może to kwestia ustalonego random_state.

### AoE 2

In [9]:
model.fit(X_train_aoe2, y_train_aoe2)
y_predicted_aoe2 = model.predict(X_test_aoe2)
accuracy_score(y_test_aoe2, y_predicted_aoe2)

0.5066666666666667

#### Strojenie


In [46]:
grid = RandomizedSearchCV(model, hyperF, cv = 11, random_state = 11)
grid.fit(X_train_aoe2, y_train_aoe2)
grid.best_params_

{'gamma': 0.22999999999999995, 'degree': 2, 'C': 4.9}

In [47]:
svm_aoe2_strj = svm.SVC(gamma = 0.22999999999999995, degree = 2, C = 4.9)
svm_aoe2_strj.fit(X_train_aoe2, y_train_aoe2)
y_predicted_aoe2_strj = svm_aoe2_strj.predict(X_test_aoe2)
accuracy_score(y_test_ap, y_predicted_aoe2_strj)

0.0

Stwierdzenie, że losowe strojenie pogorszyło wynik byłoby tutaj niedopowiedzeniem.

## Inne jądra
### Liniowe
#### Apartments

In [12]:
model = svm.SVC(kernel='linear')
model.fit(X_train_ap, y_train_ap)
y_predicted_ap = model.predict(X_test_ap)
accuracy_score(y_test_ap, y_predicted_ap)

0.26

#### AoE2

In [ ]:
model.fit(X_train_aoe2, y_train_aoe2)
y_predicted_aoe2 = model.predict(X_test_aoe2)
accuracy_score(y_test_aoe2, y_predicted_aoe2)

Powyższy kod wykonywał się ponad pół godziny i nie wykonał się, dlatego zrezygnowałem z analizy tego jądra.

### Poly

#### Apartments

In [9]:
model = svm.SVC(kernel='poly')
model.fit(X_train_ap, y_train_ap)
y_predicted_ap = model.predict(X_test_ap)
accuracy_score(y_test_ap, y_predicted_ap)

0.24

#### AoE2

In [10]:
model.fit(X_train_aoe2, y_train_aoe2)
y_predicted_aoe2 = model.predict(X_test_aoe2)
accuracy_score(y_test_aoe2, y_predicted_aoe2)

0.5066666666666667

### Sigmoid
#### Apartments

In [11]:
model = svm.SVC(kernel='sigmoid')
model.fit(X_train_ap, y_train_ap)
y_predicted_ap = model.predict(X_test_ap)
accuracy_score(y_test_ap, y_predicted_ap)

0.0

#### AoE2

In [12]:
model.fit(X_train_aoe2, y_train_aoe2)
y_predicted_aoe2 = model.predict(X_test_aoe2)
accuracy_score(y_test_aoe2, y_predicted_aoe2)

0.5066666666666667

### Podsumowanie

Wykorzystanie innych jąder nie poprawiło zbytnio sytuacji, jeżęli wręcz jej nie pogorszyło.

## Skalowanie

### Apartments

In [31]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled_ap = scaler.fit_transform(X_ap)
X_scaled_ap = pd.DataFrame(X_scaled_ap, columns = X_ap.columns)

In [32]:
X_train_ap, X_test_ap, y_train_ap, y_test_ap = train_test_split(X_scaled_ap, y_ap, test_size=0.15, random_state=111)
y_train_ap = y_train_ap.values.ravel()
y_test_ap = y_test_ap.values.ravel()
model = svm.SVC()
model.fit(X_train_ap, y_train_ap)
y_predicted = model.predict(X_test_ap)
accuracy_score(y_test_ap, y_predicted)


0.26

#### Strojenie

In [21]:
grid = RandomizedSearchCV(model, hyperF, cv = 11, random_state = 11)
grid.fit(X_train_ap, y_train_ap)
grid.best_params_

{'gamma': 0.1, 'degree': 1, 'C': 1.2000000000000002}

In [27]:
model = svm.SVC(gamma = 0.1, C = 1.2)
model.fit(X_train_ap, y_train_ap)
y_predicted = model.predict(X_test_ap)
accuracy_score(y_test_ap, y_predicted)

0.25333333333333335

### AoE 2

In [24]:
X_scaled_aoe2 = scaler.fit_transform(X_aoe2)
X_train_aoe2, X_test_aoe2, y_train_aoe2, y_test_aoe2 = train_test_split(X_scaled_aoe2, y_aoe2, test_size=0.15, random_state=111)
y_train_aoe2 = y_train_aoe2.values.ravel()
y_test_aoe2 = y_test_aoe2.values.ravel()
model = svm.SVC()
model.fit(X_train_aoe2, y_train_aoe2)
y_predicted = model.predict(X_test_aoe2)
accuracy_score(y_test_aoe2, y_predicted)

0.6933333333333334

#### Strojenie

In [25]:
grid = RandomizedSearchCV(model, hyperF, cv = 11, random_state = 11)
grid.fit(X_train_aoe2, y_train_aoe2)
grid.best_params_

{'gamma': 'auto', 'degree': 5, 'C': 6.5}

In [26]:
model = svm.SVC(gamma = 0.1, degree = 1, C = 1.2)
model.fit(X_train_aoe2, y_train_aoe2)
y_predicted = model.predict(X_test_aoe2)
accuracy_score(y_test_aoe2, y_predicted)

0.6333333333333333

### Wnioski

Strojenie za pomocą RandomizedSearch nie poprawiło znacząco wyników (a nawet je pogorszyło względem domyślnych), za to przeskalowanie danych dało znaczące polepszenie polepszenie (z 0.2 do 0.26 i z 0.5 do 0.6 dla obu zbiorów danych). Zwrócenie uwagi na skalowanie danych jest słuszne, ponieważ może to znacznie wpłynąć na wynik.

## Oświadczenie
Potwierdzam samodzielność powyższej pracy oraz niekorzystanie przeze mnie z niedozwolonych źródeł